本文学习于[imbalanced-learn User Guide](https://imbalanced-learn.org/en/stable/introduction.html)和[Imblearn package study（不平衡数据处理之过采样、下采样、综合采样）](https://blog.csdn.net/kizgel/article/details/78553009?locationNum=6&fps=1), 编辑:Weiyang,Time:2019.2.9,weixin:damo894127201

# 准备知识

## Compressed Sparse Rows(CSR):压缩稀疏的行

Sparse input:
对于稀疏输入，数据会在灌进sampler前被转换成**压缩稀疏的行(CSR)表示**。为了避免不必要的内存拷贝，推荐将数据在程序入口用CSR表示。

稀疏矩阵中存在许多0元素，按矩阵$A$的原始形式进行存储会占用很大的内存空间。CSR方法采取**按行压缩**的办法，将原始矩阵用三个数组进行表示:

In [1]:
import numpy as np

data = np.array([1, 2, 3, 4, 5, 6])
indices = np.array([0, 2, 2, 0, 1, 2])
indptr = np.array([0, 2, 3, 6])

1. data数组:存储着矩阵$A$中所有的非零元素
2. indices数组:data数组中的元素在矩阵$A$列索引
3. indptr数组:用来推断矩阵$A$中每行非零元素的列索引以及其值
4. csr_matrix((data,indices,indptr),shape=(m,n))是标准的CSR表示，对于第$i$行来说，它的非零元素的列索引存储在indices[indptr[i]:indptr[i+1]],相应的值存储在data[indptr[i]:indptr[i+1]]。如果shape参数未给定，重构的矩阵维度将会自动推断

根据上述信息重构矩阵，维度自动推断

In [2]:
from scipy import sparse

matrix = sparse.csr_matrix((data,indices,indptr))
matrix.todense()

matrix([[1, 0, 2],
        [0, 0, 3],
        [4, 5, 6]])

为什么会有针对不平衡数据的研究呢？当我们的样本数据中，正负样本的数据占比极其不均衡的时候，模型的效果就会偏向于多数类。

# 过采样(Over-sampling)

## 朴素随机过采样

针对不平衡数据，最简单的方法是**生成少数类的样本**，实现它的最基本的方法便是从少数类的样本中进行随机采样来增加新的样本，**$RandomOverSampler$**函数就实现了此功能。

用sklearn.datasets.make_classification来生成一些分类的伪数据

In [3]:
from sklearn.datasets import make_classification
from collections import Counter

X,Y = make_classification(n_samples=5000, # 生成的数据个数
                          n_features=3, # 特征个数=n_informative+n_redundant+n_repeated
                          n_informative=3,#具备独立信息的特征个数
                          n_redundant=0,# 冗余特征的个数，冗余特征是informative特征的随机线性组合
                          n_repeated=0, # 重复特征的个数，它是随机提取n_informative和n_redundant特征
                          n_classes=3, # 类别个数
                          n_clusters_per_class=2,#每个类别有几个聚类中心
                          weights=[0.01,0.05,0.94],# 每个类别占有的数据量比例
                          flip_y=0.01,
                          class_sep=1.0,# 乘以超立方体大小的因子。较大的值分散了簇/类，并使分类任务更容易。
                          hypercube=True,
                          shift=0.0,
                          scale=1.0,
                          shuffle=True,# 数据混洗
                          random_state=0
                         )

make_classification()返回值:
1. X:数组shape为[n_samples，n_features]，它是生成的样本
2. Y:数组shape为[n_samples]，它是每个样本的类成员的整数标签

In [4]:
print(X.shape,Y.shape)

(5000, 3) (5000,)


统计各类别的样本数量

In [5]:
Counter(Y)

Counter({0: 69, 1: 266, 2: 4665})

对少数类进行朴素随机过采样

In [6]:
from imblearn.over_sampling import RandomOverSampler

sampler = RandomOverSampler() # 采样器
X_resampled,Y_resampled = sampler.fit_sample(X,Y) # 随机采样

采样结果,各类别比例1:1:1

In [7]:
print(X_resampled.shape,Y_resampled.shape)

(13995, 3) (13995,)


In [8]:
Counter(Y_resampled)

Counter({0: 4665, 1: 4665, 2: 4665})

## 从随机过采样到SOMOTE与ADASYN

相对于朴素随机过采样的方法, 还有两种比较流行的采样少数类的方法:
1. Synthetic Minority Oversampling Technique (SMOTE)
2. Adaptive Synthetic (ADASYN)

SMOTE: 对于少数类样本a, 随机选择一个最近邻的样本b, 然后从a与b的连线上随机选取一个点c作为新的少数类样本
ADASYN: 关注的是在那些基于K最近邻分类器被错误分类的原始样本附近生成新的少数类样本

### SMOTE

In [9]:
from imblearn.over_sampling import SMOTE,ADASYN

X_resampled_SMOTE,Y_resampled_SMOTE = SMOTE().fit_sample(X,Y)

采样结果

In [10]:
print(X_resampled_SMOTE.shape,Y_resampled_SMOTE.shape)

(13995, 3) (13995,)


In [11]:
Counter(Y_resampled_SMOTE)

Counter({0: 4665, 1: 4665, 2: 4665})

### ADASYN

In [12]:
from imblearn.over_sampling import SMOTE,ADASYN

X_resampled_SMOTE,Y_resampled_SMOTE = ADASYN().fit_sample(X,Y)

print(X_resampled_SMOTE.shape,Y_resampled_SMOTE.shape)
Counter(Y_resampled_SMOTE)

(13992, 3) (13992,)


Counter({0: 4680, 1: 4647, 2: 4665})

## SMOTE的变体：borderline-1,borderline-2,svm

相对于基本的SMOTE算法关注的是所有的少数类样本, 这些情况可能会导致产生次优的决策函数, 因此SMOTE就产生了一些变体: 这些方法关注在最优化决策函数边界的一些少数类样本, 然后在最近邻类的相反方向生成样本.

SMOTE函数中的kind参数控制了选择哪种变体:
1. borderline1
2. borderline2
3. svm

### borderline-1

In [13]:
from imblearn.over_sampling import SMOTE,ADASYN

X_resampled,Y_resampled = SMOTE(kind='borderline1').fit_sample(X,Y)

采样结果

In [14]:
print(X_resampled.shape,Y_resampled.shape)
Counter(Y_resampled)

(13995, 3) (13995,)


Counter({0: 4665, 1: 4665, 2: 4665})

### borderline-2

In [15]:
from imblearn.over_sampling import SMOTE,ADASYN

X_resampled,Y_resampled = SMOTE(kind='borderline2').fit_sample(X,Y)

print(X_resampled.shape,Y_resampled.shape)
Counter(Y_resampled)

(13994, 3) (13994,)


Counter({0: 4664, 1: 4665, 2: 4665})

### svm

In [16]:
from imblearn.over_sampling import SMOTE,ADASYN

X_resampled,Y_resampled = SMOTE(kind='svm').fit_sample(X,Y)

print(X_resampled.shape,Y_resampled.shape)
Counter(Y_resampled)

(13995, 3) (13995,)


Counter({0: 4665, 1: 4665, 2: 4665})

## SMOTE，ADASYN，borderline-1,borderline-2,svm比较

**SMOTE算法**与**ADASYN**都是基于同样的算法来合成新的少数类样本: 对于少数类样本a, 从它的最近邻中选择一个样本b, 然后在两点的连线上随机生成一个新的少数类样本, 不同的是对于少数类样本的选择.

原始的SMOTE: kind='regular' , 随机选取少数类的样本.

The borderline SMOTE: kind='borderline1' or kind='borderline2'

此时, 少数类的样本分为三类: 
1. 噪音样本(noise), 该少数类的所有最近邻样本都来自于不同于样本a的其他类别
2. 危险样本(in danger), 至少一半的最近邻样本来自于同一类(不同于a的类别)
3. 安全样本(safe), 所有的最近邻样本都来自于同一个类

这两种类型的SMOTE使用的是**危险样本**来生成新的样本数据:
1. 对于 Borderline-1 SMOTE, 最近邻中的随机样本b与该少数类样本a来自于不同的类; 
2. 对于 Borderline-2 SMOTE , 随机样本b可以是属于任何一个类的样本;

SVM SMOTE: kind='svm', 使用支持向量机分类器产生支持向量然后再生成新的少数类样本.

# 欠采样(下采样Under-sampling)

## 原型生成算法(prototype generation)

给定数据集$S$, 原型生成算法将生成一个子集$S’$, 其中$|S’| < |S|$, 但是**子集并非来自于原始数据集**. 意思就是说: 原型生成方法将减少数据集的样本数量, 新样本是由原始数据集生成的, 而不是直接来源于原始数据集。

**ClusterCentroids函数**实现了上述功能: 每一个类别的样本都会用K-Means算法的中心点来进行合成, 而不是随机从原始样本进行抽取。

In [17]:
from imblearn.under_sampling import ClusterCentroids

sampler = ClusterCentroids()
X_resampled,Y_resampled = sampler.fit_sample(X,Y)

Counter(Y_resampled)

Counter({0: 69, 1: 69, 2: 69})

ClusterCentroids函数提供了一种很高效的方法来减少样本的数量, 但需要注意的是, 该方法要求原始数据集最好能聚类成簇。 此外, 中心点的数量应该设置好, 这样下采样的簇能很好地代表原始数据。

## 原型选择算法(prototype selection)

与原型生成不同的是, 原型选择算法是直接从原始数据集中进行抽取. 抽取的方法大概可以分为两类: 
1. 可控的下采样技术(The controlled under-sampling techniques)
2. 下采样清洗技术(the cleaning under-sampling techniques)

**可控的下采样技术**可以由用户指定下采样抽取的子集中样本的数量; 下采样清洗技术则不接受这种用户的干预。

###  Controlled under-sampling techniques：可控的下采样技术

#### 朴素的下采样技术

**RandomUnderSampler**函数是一种快速并十分简单的方式来平衡各个类别的数据: 随机选取数据的子集。

In [18]:
from imblearn.under_sampling import RandomUnderSampler

sampler = RandomUnderSampler(random_state=0) # 默认重复采样
X_resampled,Y_resampled = sampler.fit_sample(X,Y)

print(X_resampled.shape,Y_resampled.shape)
Counter(Y_resampled)

(207, 3) (207,)


Counter({0: 69, 1: 69, 2: 69})

通过设置RandomUnderSampler中的replacement=True参数, 可以实现不重复抽样。

In [19]:
from imblearn.under_sampling import RandomUnderSampler

sampler = RandomUnderSampler(random_state=0,replacement=True)
X_resampled,Y_resampled = sampler.fit_sample(X,Y)

print(X_resampled.shape,Y_resampled.shape)
Counter(Y_resampled)

(207, 3) (207,)


Counter({0: 69, 1: 69, 2: 69})

看看采样是否重复

In [20]:
import numpy as np

np.vstack({tuple(row) for row in X_resampled}).shape# set的列表推断,达到去重的目的

/home/weiyang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  This is separate from the ipykernel package so we can avoid doing imports until


(195, 3)

195 < 207 由此可见，replacement=True是不重复采样

#### Near Miss方法

NearMiss函数则添加了一些启发式(heuristic)的规则来选择样本, 通过设定version参数来实现三种启发式的规则。下面通过一个例子来说明这三个启发式的选择样本的规则, 首先我们假设正样本是需要下采样的(多数类样本), 负样本是少数类的样本.

1. NearMiss-1: 选择离N个最近邻的负样本的平均距离最小的正样本
2. NearMiss-2: 选择离N个最远的负样本的平均距离最小的正样本
3. NearMiss-3: 是一个两段式的算法。首先, 对于每一个负样本, 保留它们的M个近邻样本; 接着, 那些到N个近邻样本平均距离最大的正样本将被选择

In [21]:
from imblearn.under_sampling import NearMiss

sampler = NearMiss(random_state=0,version=1) # version=1,2,3
X_resampled,Y_resampled = sampler.fit_sample(X,Y)

print(X_resampled.shape,Y_resampled.shape)
Counter(Y_resampled)

(207, 3) (207,)


Counter({0: 69, 1: 69, 2: 69})

### Cleaning under-sampling techniques：下采样清洗技术

#### Tomek’s links

TomekLinks : 样本x与样本y来自于不同的类别, 满足以下条件, 它们之间被称之为TomekLinks:
![tomek_links](./image/tomek_links.png)

1. 不存在另外一个样本z, 使得d(x,z) < d(x,y) 或者 d(y,z) < d(x,y)成立
2. 其中d(.)表示两个样本之间的距离, 也就是说两个样本之间互为最近邻关系
3. 这个时候, 样本x或样本y很有可能是噪声数据, 或者两个样本在边界的位置附近

TomekLinks函数中的sampling_strategy参数控制Tomek’s links中的哪些样本被剔除:
1. 当sampling_strategy为str时，指定重采样的类别。注意被采样的类别数据数量要不一致，可选的参数有:
    1. 默认的sampling_strategy='auto' 移除多数类的样本 
    2. 当sampling_strategy='all'时, 多数类和少数类样本均被移除
    3. 当sampling_strategy='majority'时,只对大多数类采样
    4. 当sampling_strategy='not minority'时,对其他所有类别重采样，除了数量最少的类别
    5. 当sampling_strategy='not majority'时，对其他所有类别重采样，除了数量最多的类别
2. 当sampling_strategy为list时，list中包含被采样的类别

![tomek_links_2](./image/tomek_links_2.png)

In [22]:
from imblearn.under_sampling import TomekLinks

sampler = TomekLinks(sampling_strategy='auto')
X_resampled,Y_resampled = sampler.fit_sample(X,Y)

print(X_resampled.shape,Y_resampled.shape)
Counter(Y_resampled)

(4865, 3) (4865,)


Counter({0: 69, 1: 206, 2: 4590})

#### EditedNearestNeighbours :Edited data set using nearest neighbours(最近邻规则或ENN)

**EditedNearestNeighbours**这种方法应用**最近邻算法来编辑(edit)数据集**, 找出那些**与邻居不太友好的样本**然后移除。对于每一个要进行下采样的样本, 那些不满足一些准则的样本将会被移除:他们的绝大多(kind_sel='mode')或者全部(kind_sel='all')的近邻样本都属于同一个类, 这些样本会被保留在数据集中。

**EditedNearestNeighbours(sampling_strategy,kind_sel)**算法
1. sampling_strategy同上
2. kind_sel='all'或者'mode'

In [23]:
from imblearn.under_sampling import EditedNearestNeighbours

sampler = EditedNearestNeighbours(random_state=0,kind_sel='all')
X_resampled,Y_resampled = sampler.fit_sample(X,Y)

print(X_resampled.shape,Y_resampled.shape)
Counter(Y_resampled)

(4509, 3) (4509,)


Counter({0: 69, 1: 108, 2: 4332})

#### RepeatedEditedNearestNeighbours

**RepeatedEditedNearestNeighbours(sampling_strategy,kind_sel,max_iter)**算法
1. 算法目的是重复EditedNearestNeighbours算法多次
2. 参数sampling_strategy,kind_sel含义同上
3. max_iter，单轮最大迭代次数

In [24]:
from imblearn.under_sampling import RepeatedEditedNearestNeighbours

sampler = RepeatedEditedNearestNeighbours(random_state=0,max_iter=100)
X_resampled,Y_resampled = sampler.fit_sample(X,Y)

print(X_resampled.shape,Y_resampled.shape)
Counter(Y_resampled)

(4450, 3) (4450,)


Counter({0: 69, 1: 96, 2: 4285})

#### AllKNN

**AllKNN()**算法
1. 与RepeatedEditedNearestNeighbours算法不同的是, ALLKNN算法在进行每次迭代的时候, 最近邻的数量都在增加

In [25]:
from imblearn.under_sampling import AllKNN

sampler = AllKNN(random_state=0)
X_resampled,Y_resampled = sampler.fit_sample(X,Y)

print(X_resampled.shape,Y_resampled.shape)
Counter(Y_resampled)

(4609, 3) (4609,)


Counter({0: 69, 1: 113, 2: 4427})

#### CondensedNearestNeighbour: Condensed nearest neighbors and derived algorithms

CondensedNearestNeighbour 使用1近邻的方法来进行迭代, 来判断一个样本是应该保留还是剔除, 具体的实现步骤如下:
1. 集合C: 所有的少数类样本
2. 选择一个多数类样本(需要下采样)加入集合C, 其他的这类样本放入集合S
3. 使用集合S训练一个1-NN的分类器, 对集合S中的样本进行分类(能够被正确分类的样本是无用的，因为有其它样本已经包含了该样本的信息)
4. 将集合S中错分的样本加入集合C
5. 重复上述过程, 直到没有样本再加入到集合C
6. 集合C便是保留的新样本集合，其它样本删除即可

In [26]:
from imblearn.under_sampling import CondensedNearestNeighbour

sampler = CondensedNearestNeighbour(random_state=0)
X_resampled,Y_resampled = sampler.fit_sample(X,Y)

print(X_resampled.shape,Y_resampled.shape)
Counter(Y_resampled)

(367, 3) (367,)


Counter({0: 69, 1: 54, 2: 244})

显然, CondensedNearestNeighbour方法**对噪音数据是很敏感的**, 也容易加入噪音数据到集合C中。

#### OneSidedSelection

OneSidedSelection 函数使用 TomekLinks 方法来剔除噪声数据(多数类样本)

In [27]:
from imblearn.under_sampling import OneSidedSelection

sampler = OneSidedSelection(random_state=0)
X_resampled,Y_resampled = sampler.fit_sample(X,Y)

print(X_resampled.shape,Y_resampled.shape)
Counter(Y_resampled)

(4463, 3) (4463,)


Counter({0: 69, 1: 168, 2: 4226})

#### NeighbourhoodCleaningRule ：邻域清理规则，NCL

NeighbourhoodCleaningRule 算法主要关注如何清洗数据而不是筛选他们。
因此, 该算法将使用EditedNearestNeighbours和 3-NN分类器结果拒绝的样本之间的并集。

主要思想:
针对训练样本集中的每个样本找出其三个最近邻样本，若该样本是多数类样本且其三个最近邻中有两个以上是少数类样本，则删除它；反之当该样本是少数类并且其三个最近邻中有两个以上是多数类样本，则去除近邻中的多数类样本。

缺陷:
未能考虑到在少数类样本中存在的噪声样本而且第二种方法删除的多数类样本大多属于边界样本，删除这些样本，对后续分类器的分类产生很大的不良影响。

In [28]:
from imblearn.under_sampling import NeighbourhoodCleaningRule

sampler = NeighbourhoodCleaningRule(random_state=0)
X_resampled,Y_resampled = sampler.fit_sample(X,Y)

print(X_resampled.shape,Y_resampled.shape)
Counter(Y_resampled)

(4820, 3) (4820,)


Counter({0: 69, 1: 157, 2: 4594})

#### InstanceHardnessThreshold: Instance hardness threshold

InstanceHardnessThreshold是在数据上运用一种**分类器**, 然后将概率低于阈值的样本剔除掉

In [29]:
from sklearn.linear_model import LogisticRegression
from imblearn.under_sampling import InstanceHardnessThreshold

sampler = InstanceHardnessThreshold(random_state=0,estimator=LogisticRegression())
X_resampled,Y_resampled = sampler.fit_sample(X,Y)

print(X_resampled.shape,Y_resampled.shape)
Counter(Y_resampled)

(207, 3) (207,)


/home/weiyang/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/weiyang/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/weiyang/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/weiyang/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/weiyang/anaconda3/lib/python3.6/site-packages/sklearn/linear_mod

Counter({0: 69, 1: 69, 2: 69})

# 过采样和欠采样的结合

在之前的SMOTE方法中, 当由边界的样本与其他样本进行过采样线性插值时, 很容易生成一些噪音数据. 因此, 在过采样之后需要对样本进行清洗. 这样, 第三节中涉及到的**TomekLink** 与 **EditedNearestNeighbours**方法就能实现上述的要求. 所以就有了两种结合过采样与下采样的方法: (i) SMOTETomek and (ii) SMOTEENN

## SMOTE+ENN

In [30]:
from imblearn.combine import SMOTEENN

sampler = SMOTEENN(random_state=0)
X_resampled,Y_resampled = sampler.fit_sample(X,Y)

print(X_resampled.shape,Y_resampled.shape)
Counter(Y_resampled)

(11824, 3) (11824,)


Counter({0: 4190, 1: 4257, 2: 3377})

## SMOTE+TomekLink

In [31]:
from imblearn.combine import SMOTETomek

sampler = SMOTETomek(random_state=0)
X_resampled,Y_resampled = sampler.fit_sample(X,Y)

print(X_resampled.shape,Y_resampled.shape)
Counter(Y_resampled)

(13671, 3) (13671,)


Counter({0: 4598, 1: 4567, 2: 4506})

# Ensemble:数据集的集成

一个不均衡的数据集能够通过多个均衡的子集来实现均衡

## EasyEnsemble

一个不均衡的数据集能够通过多个均衡的子集来实现均衡, imblearn.ensemble模块能实现上述功能。

EasyEnsemble 通过对原始的数据集进行随机下采样实现对数据集进行集成。

In [32]:
from imblearn.ensemble import EasyEnsemble

sampler = EasyEnsemble(random_state=0,n_subsets=10)
X_resampled,Y_resampled = sampler.fit_sample(X,Y)

print(X_resampled.shape,Y_resampled.shape)
Counter(Y_resampled[0]) # 取一个子集看看各类的比例

(10, 207, 3) (10, 207)


/home/weiyang/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class EasyEnsemble is deprecated; EasyEnsemble is deprecated in 0.4 and will be removed in 0.6. Use EasyEnsembleClassifier instead.
  warnings.warn(msg, category=DeprecationWarning)


Counter({0: 69, 1: 69, 2: 69})

EasyEnsemble 有两个很重要的参数: 
1. n_subsets 控制的是子集的个数  
2. replacement 决定是有放回还是无放回的随机采样

## BalanceCascade:级联平衡

BalanceCascade(级联平衡)的方法通过使用**分类器(estimator参数)**来确保那些**被错分类的样本**在下一次进行子集选取的时候也能被采样到. 同样, n_max_subset 参数控制子集的个数, 以及可以通过设置bootstrap=True来使用bootstraping(自助法)

In [33]:
from imblearn.ensemble import BalanceCascade
from sklearn.linear_model import LogisticRegression

sampler = BalanceCascade(random_state=0,
                         estimator=LogisticRegression(random_state=0),
                         n_max_subset=3
                        )
X_resampled,Y_resampled = sampler.fit_sample(X,Y)

print(X_resampled.shape,Y_resampled.shape)
Counter(Y_resampled[0]) #取一个子集看看各类的比例

(3, 207, 3) (3, 207)


/home/weiyang/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class BalanceCascade is deprecated; BalanceCascade is deprecated in 0.4 and will be removed in 0.6.
  warnings.warn(msg, category=DeprecationWarning)
/home/weiyang/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/weiyang/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/weiyang/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/weiyang/anaconda3/lib/python3.6/site-packages/sklearn/linea

Counter({0: 69, 1: 69, 2: 69})

## Chaining ensemble of samplers and estimators：Bagging

在集成分类器中, 装袋方法(Bagging)在**不同的随机选取的数据集**上建立了多个估计量. 在scikit-learn中这个分类器叫做**BaggingClassifier**. 然而, 该分类器并不允许对每个数据集进行均衡. 因此, 在对不均衡样本进行训练的时候, 分类器其实是有偏的, 偏向于多数类。

### 在sklearn上使用**BaggingClassifier**

In [34]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

X_train,X_test,Y_train,Y_test = train_test_split(X,Y,random_state=0)
model = BaggingClassifier(base_estimator=DecisionTreeClassifier(),random_state=0)
model.fit(X_train,Y_train)
Y_pred = model.predict(X_test)
confusion_matrix(Y_test,Y_pred)

array([[   6,    4,   12],
       [   1,   42,   31],
       [   3,   11, 1140]])

### BalancedBaggingClassifier

BalancedBaggingClassifier 允许在训练每个基学习器之前对每个子集进行重抽样. 简而言之, 该方法结合了EasyEnsemble 采样器与分类器(如BaggingClassifier)的结果。

In [35]:
from imblearn.ensemble import BalancedBaggingClassifier

model = BalancedBaggingClassifier(base_estimator=DecisionTreeClassifier(),
                                  ratio='auto',
                                  replacement=False,
                                  random_state=0
                                 )
model.fit(X_train,Y_train)
Y_pred = model.predict(X_test)
confusion_matrix(Y_test,Y_pred)

array([[  8,   7,   7],
       [ 13,  54,   7],
       [164, 121, 869]])

# 数据载入

**imblearn.datasets** 包与**sklearn.datasets** 包形成了很好的互补. 该包主要有以下两个功能: 
1. 提供一系列的不平衡数据集来实现测试
2. 提供一种工具将原始的平衡数据转换为不平衡数据

## 不平衡数据集:fetch_datasets

fetch_datasets 允许获取27个不均衡且二值化的数据集

```Python
from collections import Counter
from imblearn.datasets import fetch_datasets

ecoli = fetch_datasets()['ecoli']
ecoli.data.shape
Counter(ecoli.target)
```

## 生成不平衡数据:make_imblance

make_imbalance 方法可以使得原始的数据集变为不平衡的数据集, 主要是通过ratio参数进行控制

In [36]:
from sklearn.datasets import load_iris
from imblearn.datasets import make_imbalance

iris = load_iris()
ratio = {0: 20, 1: 30, 2: 40} # 各类别导入的数据量
X_imb, y_imb = make_imbalance(iris.data, iris.target, ratio=ratio)

sorted(Counter(y_imb).items())

/home/weiyang/anaconda3/lib/python3.6/site-packages/imblearn/datasets/_imbalance.py:105: UserWarning: 'ratio' has been deprecated in 0.4 and will be removed in 0.6. Use 'sampling_strategy' instead.
  warnings.warn("'ratio' has been deprecated in 0.4 and will be "


[(0, 20), (1, 30), (2, 40)]

In [37]:
#当类别不指定时, 所有的数据集均导入
ratio = {0: 10} # 类别0导入10条，其他各类全都导入
X_imb, y_imb = make_imbalance(iris.data, iris.target, ratio=ratio)

sorted(Counter(y_imb).items())

/home/weiyang/anaconda3/lib/python3.6/site-packages/imblearn/datasets/_imbalance.py:105: UserWarning: 'ratio' has been deprecated in 0.4 and will be removed in 0.6. Use 'sampling_strategy' instead.
  warnings.warn("'ratio' has been deprecated in 0.4 and will be "


[(0, 10), (1, 50), (2, 50)]

In [38]:
#同样亦可以传入自定义的比例函数
def ratio_multiplier(y):
    multiplier = {0: 0.5, 1: 0.7, 2: 0.95} # 各类别导入各自数据量的比例
    target_stats = Counter(y)
    for key, value in target_stats.items():
        target_stats[key] = int(value * multiplier[key])
    return target_stats
X_imb, y_imb = make_imbalance(iris.data, iris.target,
                              ratio=ratio_multiplier)

sorted(Counter(y_imb).items())

/home/weiyang/anaconda3/lib/python3.6/site-packages/imblearn/datasets/_imbalance.py:105: UserWarning: 'ratio' has been deprecated in 0.4 and will be removed in 0.6. Use 'sampling_strategy' instead.
  warnings.warn("'ratio' has been deprecated in 0.4 and will be "


[(0, 25), (1, 35), (2, 47)]